<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [34]:
# librerias

import pandas as pd
import numpy as np
import bayes

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem,beta

import pylab as plt

In [17]:
# datos
data = pd.read_csv('data/cookie_cats.csv')
data.head()
# data['userid'].duplicated().sum() == 0

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [102]:
th = data[data.version == 'gate_30']
fo = data[data.version == 'gate_40']
nth = th[th['sum_gamerounds']>29]
nfo = fo[fo['sum_gamerounds']>29]
cth = th[th['sum_gamerounds']>39]
cfo = fo[fo['sum_gamerounds']>39]
sth = th[th['sum_gamerounds']>1000]
sfo = fo[fo['sum_gamerounds']>1000]

In [103]:
var_dic = {'th':[th], 'fo':[fo] , 'nth':[nth], 'nfo':[nfo] , 'cfo' : [cfo] , 'cth' : [cth], 'sth' : [sth], 'sfo' : [sfo]}
beta_var = {}
betas = {}
for key,val in var_dic.items():
    df = val[0]
    var_dic[key].append(len(df))
    var_dic[key].append(sum(df[df['retention_1']]['retention_1']))
    var_dic[key].append(sum(df[df['retention_7']]['retention_7']))
    beta_var[key] = [(1 + var_dic[key][2], 1 + var_dic[key][1] - var_dic[key][2]),
                     (1 + var_dic[key][3], 1 + var_dic[key][1] - var_dic[key][3])]
    betas[key] = (beta(*(beta_var[key][0])),beta(*(beta_var[key][1])))

In [104]:
print('control (all), test (all), 1 day: ',1-bayes.calc_prob(betas['th'][0],betas['fo'][0]))
print('means: ',round(betas['th'][0].mean(),4),' ',round(betas['fo'][0].mean(),4))
print('control (all), test (all), 7 day: ',1-bayes.calc_prob(betas['th'][1],betas['fo'][1]))
print('means: ',round(betas['th'][1].mean(),4),' ',round(betas['fo'][1].mean(),4))
print('control (>29), test (>29), 1 day: ',1-bayes.calc_prob(betas['nth'][0],betas['nfo'][0]))
print('means: ',round(betas['nth'][0].mean(),4),' ',round(betas['nfo'][0].mean(),4))
print('control (>29), test (>29), 7 day: ',1-bayes.calc_prob(betas['nth'][1],betas['nfo'][1]))
print('means: ',round(betas['nth'][1].mean(),4),' ',round(betas['nfo'][1].mean(),4))
print('control (>29), test (>39), 1 day: ',1-bayes.calc_prob(betas['nth'][0],betas['cfo'][0]))
print('means: ',round(betas['nth'][0].mean(),4),' ',round(betas['cfo'][0].mean(),4))
print('control (>29), test (>39), 7 day: ',1-bayes.calc_prob(betas['nth'][1],betas['cfo'][1]))
print('means: ',round(betas['nth'][1].mean(),4),' ',round(betas['cfo'][1].mean(),4))
print('control (>39), test (>39), 1 day: ',1-bayes.calc_prob(betas['cth'][0],betas['cfo'][0]))
print('means: ',round(betas['cth'][0].mean(),4),' ',round(betas['cfo'][0].mean(),4))
print('control (>39), test (>39), 7 day: ',1-bayes.calc_prob(betas['cth'][1],betas['cfo'][1]))
print('means: ',round(betas['cth'][1].mean(),4),' ',round(betas['cfo'][1].mean(),4))
print('control (>1000), test (>1000), 1 day: ',1-bayes.calc_prob(betas['sth'][0],betas['sfo'][0]))
print('means: ',round(betas['sth'][0].mean(),4),' ',round(betas['sfo'][0].mean(),4))
print('control (>1000), test (>1000), 7 day: ',1-bayes.calc_prob(betas['sth'][1],betas['sfo'][1]))
print('means: ',round(betas['sth'][1].mean(),4),' ',round(betas['sfo'][1].mean(),4))

control (all), test (all), 1 day:  0.037206025148464605
means:  0.4482   0.4423
control (all), test (all), 7 day:  0.0007773386764495793
means:  0.1902   0.182
control (>29), test (>29), 1 day:  0.5787503124298948
means:  0.8009   0.8018
control (>29), test (>29), 7 day:  0.05399420505314878
means:  0.4388   0.43
control (>29), test (>39), 1 day:  0.9999999999697599
means:  0.8009   0.8301
control (>29), test (>39), 7 day:  0.9999999999999997
means:  0.4388   0.4851
control (>39), test (>39), 1 day:  0.0494914745217665
means:  0.8376   0.8301
control (>39), test (>39), 7 day:  0.0016010147332363722
means:  0.5029   0.4851
control (>1000), test (>1000), 1 day:  0.7372528714367622
means:  0.9273   0.9538
control (>1000), test (>1000), 7 day:  0.15609405206139615
means:  0.9818   0.9538


In [113]:
def fn(n):
    a = th[th['sum_gamerounds']>n]
    b = fo[fo['sum_gamerounds']>n]
    
    alen = len(a)
    blen = len(b)
    a1 = len(a[a['retention_1']])
    b1 = len(b[b['retention_1']])
    a7 = len(a[a['retention_7']])
    b7 = len(b[b['retention_7']])
    abeta1 = beta(1 + a1 , 1 + alen - a1)
    bbeta1 = beta(1 + b1 , 1 + blen - b1)
    abeta7 = beta(1 + a7 , 1 + alen - a7)
    bbeta7 = beta(1 + b7 , 1 + blen - b7)
    print(f'control, test, 1 day, >{n} games: ',round(1-bayes.calc_prob(abeta1,bbeta1),4))       
    print('means: ',round(abeta1.mean(),4),' ',round(bbeta1.mean(),4), ' ',round(bbeta1.mean() - abeta1.mean(),4))
    print()
    print(f'control, test, 7 day, >{n} games: ',round(1-bayes.calc_prob(abeta7,bbeta7),4))       
    print('means: ',round(abeta7.mean(),4),' ',round(bbeta7.mean(),4), ' ',round(bbeta7.mean()-abeta7.mean(),4))
    print()
    print('=====================================')
    print()

In [116]:
'''for i in range(1,5000,50):
    fn(i)'''

control, test, 1 day, >1 games:  0.0541
means:  0.4975   0.4918   -0.0057

control, test, 7 day, >1 games:  0.0011
means:  0.2116   0.2029   -0.0087


control, test, 1 day, >51 games:  0.0173
means:  0.8625   0.8527   -0.0099

control, test, 7 day, >51 games:  0.0001
means:  0.5675   0.5425   -0.025


control, test, 1 day, >101 games:  0.2927
means:  0.9019   0.8989   -0.0029

control, test, 7 day, >101 games:  0.443
means:  0.7163   0.7152   -0.0012


control, test, 1 day, >151 games:  0.6068
means:  0.9209   0.9226   0.0017

control, test, 7 day, >151 games:  0.6074
means:  0.7968   0.7993   0.0025


control, test, 1 day, >201 games:  0.3128
means:  0.9363   0.933   -0.0033

control, test, 7 day, >201 games:  0.558
means:  0.8499   0.8514   0.0014


control, test, 1 day, >251 games:  0.4789
means:  0.9427   0.9423   -0.0004

control, test, 7 day, >251 games:  0.8068
means:  0.8838   0.8928   0.009


control, test, 1 day, >301 games:  0.6939
means:  0.9449   0.9493   0.0044

control, 

### Frequentist

z_score, p_value = 